In [1]:
import math
import random

from hypothesis import given
import hypothesis.strategies as st

# Closest pair of points problem

Problem is two find the minimum distance between two different points in a collection of points.  A naive solution to this problem would enumerate all possible pairs of points, compute their distances, and then find the minimum. This naive solution runs in $O(n^2)$ time.  Here is a naive implementation.

In [2]:
def _euclidean_distance(p1, p2):
    d = ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**0.5
    return d


def brute_force_minimum_distance(points):
    """Brute force algorithm for finding minimum distance is O(n^2)."""
    if len(points) < 2:
        return float("inf")
    else:
        n = len(points)
        distances = []
        for i, p1 in enumerate(points):
            for j, p2 in enumerate(points):
                if i != j:
                    d = _euclidean_distance(p1, p2)
                    distances.append(d)
        return min(distances)


There is a better alternative! In particular there is an $O(n \log n)$ algorithm for finding the minimum distance between two different points! Wikipedia has a nice description of the [closest pair of points problem](https://en.m.wikipedia.org/wiki/Closest_pair_of_points_problem?) and this efficient solution.  The key to an efficient solution is to figure out a way to find the minimum distance of between any two points in the "strip" between the two regions whilst doing on and $O(m)$ amount of work (where $m$ is the number of points in the "strip" region. 

In [13]:
def efficient_minimum_distance(points):
    """Efficient algorithm for finding minimum distance is O(n log n)."""
    if len(points) < 2:
        return float("inf")
    else:
        sorted_points = sorted(points, key=lambda p: (p[0], -p[1]))
        d = _minimum_distance_between(sorted_points)
        return d


def _minimum_distance_between(sorted_points):
    """Find the minimum distance between two points."""
    n = len(sorted_points)
    if n < 2:
        return float("inf")
    elif n == 2:
        p1, p2 = sorted_points
        d = _euclidean_distance(p1, p2)
        return d
    else:
        m = n // 2
        s1, s2 = sorted_points[:m], sorted_points[m:]
        d1 = _minimum_distance_between(s1)
        d2 = _minimum_distance_between(s2)
        d = min(d1, d2)

        
        median_x_coord = _find_median_x_coord(sorted_points)
        strip_points = [p for p in sorted_points  if abs(p[0] - median_x_coord) <= d]
        d_prime = float("inf")
        for i, p1 in enumerate(strip_points):
            if i + 7 < len(strip_points):
                for p2 in strip_points[i + 1:i + 7]:
                    e = _euclidean_distance(p1, p2)
                    if e < d_prime:
                        d_prime = e
            else:
                for p2 in strip_points[i + 1:]:
                    e = _euclidean_distance(p1, p2)
                    if e < d_prime:
                        d_prime = e
        return min(d, d_prime)


def _find_median_x_coord(sorted_points):
    n = len(sorted_points)
    idx = (n - 1) // 2
    if n % 2 == 1:
        x, _ = sorted_points[idx]
        return x
    else:
        x1, _ = sorted_points[idx]
        x2, _ = sorted_points[idx + 1]
        return (x1 + x2) / 2.0



In [14]:
@given(st.lists(st.tuples(st.integers(), st.integers())))
def test_efficient_minimum_distance(points):
    expected_result = brute_force_minimum_distance(points)
    actual_result = efficient_minimum_distance(points)
    msg = "Points {}; expected distance {}; actual distance {}.".format(points, expected_result, actual_result)
    assert math.isclose(expected_result, actual_result), msg

In [15]:
test_efficient_minimum_distance()

In [16]:
%timeit efficient_minimum_distance([(random.randint(-1000, 1000), random.randint(-1000, 1000)) for _ in range(10000)])

183 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
